# <center> Workshop 4: Tropical Cyclones</center>

Will include: 
- Toy models and sensitivity - changing sst; warm top vs cold top vs neutral; rh scenarios ("dry high") and ("wet high")

Method: use warming fractions to perturb t by more or less relative to sst change. 



In [44]:
import xarray as xa, numpy as np
from pyPI import pi
import matplotlib.pyplot as plt
from src import utils
%matplotlib inline

In [45]:
fin="/home/lunet/gytm3/Teaching/GYP050/Public/Data/era5_tc_ilev.nc"
sample="/home/lunet/gytm3/Teaching/GYP050/Public/Data/sample_data.n

In [46]:
data=xa.open_dataset(fin)
test=xa.open_dataset(sample)
data["msl"]=data["msl"]/100. #hPa

dsst=2
dt=3
drh=0.25
nt,nlev,nr,nc=data["t"].shape
t=data["t"].values
q=data["q"].values
p=data["level"].values
sst=data["sst"].values
sst_out,t_out,rh_out,q_out=utils.gen_future_conditions(dsst,dt,drh,sst,t,q,p,nt,nlev,nr,nc)

data["t"].values=t_out-273.15
data["sst"].values=sst_out-273.15
data["q"].values=q_out

In [47]:
# (VMAX,PMIN,IFL,TO,LNB)=pi(SST,MSL,P,T,R)
result = xa.apply_ufunc(
        pi,
        data['sst'], data['msl'], data['level'], data['t'], data['q'],
        kwargs=dict(CKCD=0.9, ascent_flag=0, diss_flag=1, ptop=50, miss_handle=1),
        input_core_dims=[
            [], [], ['level', ], ['level', ], ['level', ],
        ],
        output_core_dims=[
            [], [], [], [], []
        ],
        vectorize=True
    )

#(VMAX,PMIN,IFL,TO,LNB)=pi(smean["sst"].values,smean["msl"].values/100.,pmean["level"].values,
 #                         pmean["t"].values[:],pmean["q"].values)

In [1]:
vmax, pmin, ifl, t0, otl = result
vmax=vmax.values.flatten()
fig,ax=plt.subplots(1,1)
h=ax.hist(vmax[~np.isnan(vmax)],density=True,bins=30)

NameError: name 'result' is not defined

In [49]:
np.percentile(vmax,99.9)

118.01870727476026